In [1]:
using SymPy
using LinearAlgebra
import DifferentialEquations
@vars J1xx J1yy J1zz J2xx J2yy J2zz l1 L1 l2 m1 m2 g b1 b2 t thetadot1 thetadot2 thetaddot1 thetaddot2
;

In [2]:
theta1 = SymFunction("theta1")(t)
theta2 = SymFunction("theta2")(t);

In [3]:
J1 = [J1xx 0 0; 0 J1yy 0; 0 0 J1zz]
J2 = [J2xx 0 0; 0 J2yy 0; 0 0 J2zz];

In [4]:
R1 = [cos(theta1) sin(theta1) 0; -sin(theta1) cos(theta1) 0; 0 0 1]
R2 = [0 sin(theta2) -cos(theta2); 0 cos(theta2) sin(theta2); 1 0 0];

In [5]:
ω1 = [0;0;diff(theta1,t)]
r1 = [l1;0;0]
v1c = cross(ω1, r1)
ω2 = R2*ω1 + [0;0;diff(theta2,t)]
v2 = R2*(cross(ω1,[L1;0;0]))
r2 = [l2;0;0]
v2c = v2 + (cross(ω2, r2));

In [6]:
v1c.subs(diff(theta1,t),thetadot1)

3×1 Array{Sym,2}:
            0
 l1*thetadot1
            0

In [7]:
v2c.subs([(diff(theta1,t),thetadot1),(diff(theta2,t),thetadot2)])

3×1 Array{Sym,2}:
                L1*thetadot1*sin(theta2(t))
 L1*thetadot1*cos(theta2(t)) + l2*thetadot2
               -l2*thetadot1*sin(theta2(t))

In [8]:
Ep1 = [0]
Ek1 = .5*((v1c.T)*m1*v1c + (ω1.T)*J1*ω1)
Ep2 = [g*m2*l2*(1-cos(theta2))]
Ek2 = .5*((v2c.T)*m2*v2c) + .5*((ω2.T)*J2*ω2)
L = Ek1 + Ek2 - Ep1 - Ep2
roughτ1 = diff((diff(L[1],diff(theta1,t))),t) + b1*(diff(theta1,t)) - diff(L[1],theta1)
cleanτ1 = roughτ1.subs([(diff(diff(theta1,t),t),thetaddot1),(diff(diff(theta2,t),t),thetaddot2),(diff(theta1,t),thetadot1),(diff(theta2,t),thetadot2)])
τ1 = collect(expand(cleanτ1),thetaddot1)

                                                                              
-2.0⋅J2xx⋅θ̇₁⋅θ̇₂⋅sin(θ₂(t))⋅cos(θ₂(t)) + 2.0⋅J2yy⋅θ̇₁⋅θ̇₂⋅sin(θ₂(t))⋅cos(θ₂(t

                                                   2                          
)) + 1.0⋅L₁⋅l₂⋅m₂⋅θ̈₂⋅cos(θ₂(t)) - 1.0⋅L₁⋅l₂⋅m₂⋅θ̇₂ ⋅sin(θ₂(t)) + b₁⋅θ̇₁ + 2.0

   2                                        ⎛                       2         
⋅l₂ ⋅m₂⋅θ̇₁⋅θ̇₂⋅sin(θ₂(t))⋅cos(θ₂(t)) + θ̈₁⋅⎝1.0⋅J1zz + 1.0⋅J2xx⋅cos (θ₂(t)) +

             2                2       2                2       2              
 1.0⋅J2yy⋅sin (θ₂(t)) + 1.0⋅L₁ ⋅m₂⋅sin (θ₂(t)) + 1.0⋅L₁ ⋅m₂⋅cos (θ₂(t)) + 1.0⋅

  2            2       2       ⎞
l₁ ⋅m₁ + 1.0⋅l₂ ⋅m₂⋅sin (θ₂(t))⎠

In [9]:
roughτ2 = diff((diff(L[1],(diff(theta2,t)))),t) + b2*(diff(theta2,t)) - diff(L[1],theta2)
cleanτ2 = roughτ2.subs([(diff(diff(theta1,t),t),thetaddot1),(diff(diff(theta2,t),t),thetaddot2),(diff(theta1,t),thetadot1),(diff(theta2,t),thetadot2)])
τ2 = collect(expand(cleanτ2),thetadot1)

                                                                              
1.0⋅J2zz⋅θ̈₂ + 1.0⋅L₁⋅l₂⋅m₂⋅θ̈₁⋅cos(θ₂(t)) + b₂⋅θ̇₂ + g⋅l₂⋅m₂⋅sin(θ₂(t)) + 1.0

   2             2 ⎛                                                          
⋅l₂ ⋅m₂⋅θ̈₂ + θ̇₁ ⋅⎝1.0⋅J2xx⋅sin(θ₂(t))⋅cos(θ₂(t)) - 1.0⋅J2yy⋅sin(θ₂(t))⋅cos(θ

              2                         ⎞
₂(t)) - 1.0⋅l₂ ⋅m₂⋅sin(θ₂(t))⋅cos(θ₂(t))⎠

In [10]:
E = Ek1 + Ek2 + Ep1 + Ep2

1-element Array{Sym,1}:
 0.5*J1zz*Derivative(theta1(t), t)^2 + 0.5*J2xx*cos(theta2(t))^2*Derivative(theta1(t), t)^2 + 0.5*J2yy*sin(theta2(t))^2*Derivative(theta1(t), t)^2 + 0.5*J2zz*Derivative(theta2(t), t)^2 + 0.5*L1^2*m2*sin(theta2(t))^2*Derivative(theta1(t), t)^2 + g*l2*m2*(1 - cos(theta2(t))) + 0.5*l1^2*m1*Derivative(theta1(t), t)^2 + 0.5*l2^2*m2*sin(theta2(t))^2*Derivative(theta1(t), t)^2 + 0.5*m2*(L1*cos(theta2(t))*Derivative(theta1(t), t) + l2*Derivative(theta2(t), t))^2

In [26]:
@vars tau1 tau2 z1 z2 z3 z4
eq1 = Eq(0,roughτ1 - tau1)

               2                                                              
              d                                          d         d          
0 = 1.0⋅J1zz⋅───(θ₁(t)) - 2.0⋅J2xx⋅sin(θ₂(t))⋅cos(θ₂(t))⋅──(θ₁(t))⋅──(θ₂(t)) +
               2                                         dt        dt         
             dt                                                               

                        2                                 2                   
             2         d                       2         d                    
 1.0⋅J2xx⋅cos (θ₂(t))⋅───(θ₁(t)) + 1.0⋅J2yy⋅sin (θ₂(t))⋅───(θ₁(t)) + 2.0⋅J2yy⋅
                        2                                 2                   
                      dt                                dt                    

                                                                     2        
                      d         d                 2       2         d         
sin(θ₂(t))⋅cos(θ₂(t))⋅──(θ₁(t))⋅──(θ₂(t)) + 1.0⋅L₁

In [27]:
eq2 = Eq(0,roughτ2 - tau2)

                                              2                               
                                   ⎛d        ⎞                                
0 = 1.0⋅J2xx⋅sin(θ₂(t))⋅cos(θ₂(t))⋅⎜──(θ₁(t))⎟  - 1.0⋅J2yy⋅sin(θ₂(t))⋅cos(θ₂(t
                                   ⎝dt       ⎠                                
                                                                              

              2              2                                                
   ⎛d        ⎞              d                 2                          ⎛d   
))⋅⎜──(θ₁(t))⎟  + 1.0⋅J2zz⋅───(θ₂(t)) - 1.0⋅L₁ ⋅m₂⋅sin(θ₂(t))⋅cos(θ₂(t))⋅⎜──(θ
   ⎝dt       ⎠               2                                           ⎝dt  
                           dt                                                 

      2                                                                       
     ⎞              ⎛              d              d        ⎞            d     
₁(t))⎟  + 1.0⋅L₁⋅m₂⋅⎜L₁⋅cos(θ₂(t))⋅──(θ₁(t)) + l₂⋅

In [42]:
variablechange = Dict(theta1=>z1,theta2=>z2,diff(theta1,t)=>z3,diff(theta2,t)=>z4)
EOM = solve([eq1, eq2],[diff(diff(theta1,t),t), diff(diff(theta2,t),t)])

Dict{Any,Any} with 2 entries:
  Derivative(theta2(t), (t, 2)) => (L1*l2*m2*(J2xx*sin(2.0*theta2(t))*Derivativ…
  Derivative(theta1(t), (t, 2)) => -(L1*l2*m2*(0.5*J2xx*sin(2.0*theta2(t))*Deri…

In [43]:
EOM1 = EOM[diff(diff(theta1,t),t)]

ErrorException: type #subs has no field variablechange

In [40]:
EOM2 = EOM[diff(diff(theta2,t),t)]

         ⎛                                                                    
         ⎜                    d         d                               d     
L₁⋅l₂⋅m₂⋅⎜J2xx⋅sin(2.0⋅θ₂(t))⋅──(θ₁(t))⋅──(θ₂(t)) - J2yy⋅sin(2.0⋅θ₂(t))⋅──(θ₁(
         ⎝                    dt        dt                              dt    
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                               2                              
    d                               ⎛d        ⎞       d             2         
t))⋅──(θ₂(t)) + L₁⋅l₂⋅m₂⋅sin(θ₂(t))⋅⎜──(θ₂(t))⎟  - b₁⋅──(θ₁(t)) - l₂ ⋅m₂⋅sin(2
    dt                              ⎝dt       ⎠       dt                      
──────────────────────────────────────────────────────────────────────────────
                                                   

In [55]:
EOM1 = EOM1.subs(variablechange)

         ⎛                                                             2      
L₁⋅l₂⋅m₂⋅⎝J2xx⋅z₃⋅z₄⋅sin(2.0⋅z₂) - J2yy⋅z₃⋅z₄⋅sin(2.0⋅z₂) + L₁⋅l₂⋅m₂⋅z₄ ⋅sin(z
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

               2                          ⎞           ⎛           2           
₂) - b₁⋅z₃ - l₂ ⋅m₂⋅z₃⋅z₄⋅sin(2.0⋅z₂) + τ₁⎠⋅cos(z₂) + ⎝0.5⋅J2xx⋅z₃ ⋅sin(2.0⋅z₂
──────────────────────────────────────────────────────────────────────────────
                           2   2   2    2       ⎛          2   ⎞ ⎛            
                         L₁ ⋅l₂ ⋅m₂ ⋅cos (z₂) + ⎝-J2zz - l₂ ⋅m₂⎠⋅⎝J1zz - J2xx⋅

               2                                               2      2       
) - 0.5⋅J2yy⋅z₃ ⋅sin(2.0⋅z₂) + b₂⋅z₄ + g⋅l₂⋅m₂⋅sin(z₂) - 0.5⋅l₂ ⋅m₂⋅z₃ ⋅sin(2.
──────────────────────────────────────────────────

In [53]:
sympy.julia_code(EOM1) |> println

-(L1.*l2.*m2.*(0.5*J2xx.*z3.^2.*sin(2.0*z2) - 0.5*J2yy.*z3.^2.*sin(2.0*z2) + b2.*z4 + g.*l2.*m2.*sin(z2) - 0.5*l2.^2.*m2.*z3.^2.*sin(2.0*z2) - tau2).*cos(z2) + (J2zz + l2.^2.*m2).*(J2xx.*z3.*z4.*sin(2.0*z2) - J2yy.*z3.*z4.*sin(2.0*z2) + L1.*l2.*m2.*z4.^2.*sin(z2) - b1.*z3 - l2.^2.*m2.*z3.*z4.*sin(2.0*z2) + tau1))./(L1.^2.*l2.^2.*m2.^2.*cos(z2).^2 - (J2zz + l2.^2.*m2).*(J1zz - J2xx.*sin(z2).^2 + J2xx + J2yy.*sin(z2).^2 + L1.^2.*m2 + l1.^2.*m1 + l2.^2.*m2.*sin(z2).^2))


In [57]:
EOM2 = EOM2.subs(variablechange)
sympy.julia_code(EOM2) |> println

(L1.*l2.*m2.*(J2xx.*z3.*z4.*sin(2.0*z2) - J2yy.*z3.*z4.*sin(2.0*z2) + L1.*l2.*m2.*z4.^2.*sin(z2) - b1.*z3 - l2.^2.*m2.*z3.*z4.*sin(2.0*z2) + tau1).*cos(z2) + (0.5*J2xx.*z3.^2.*sin(2.0*z2) - 0.5*J2yy.*z3.^2.*sin(2.0*z2) + b2.*z4 + g.*l2.*m2.*sin(z2) - 0.5*l2.^2.*m2.*z3.^2.*sin(2.0*z2) - tau2).*(J1zz - J2xx.*sin(z2).^2 + J2xx + J2yy.*sin(z2).^2 + L1.^2.*m2 + l1.^2.*m1 + l2.^2.*m2.*sin(z2).^2))./(L1.^2.*l2.^2.*m2.^2.*cos(z2).^2 + (-J2zz - l2.^2.*m2).*(J1zz - J2xx.*sin(z2).^2 + J2xx + J2yy.*sin(z2).^2 + L1.^2.*m2 + l1.^2.*m1 + l2.^2.*m2.*sin(z2).^2))
